In [1]:
import re
import os
import collections
import time

In [2]:
dictionary = {}
matrix = {}
# This is the map of docId to input file name
docIdMap = {}
docCollectionPath = "docs"
queryFile = "query"

In [3]:
def buildIndex():
    docId = 1
    fileList = [f for f in os.listdir(docCollectionPath) if os.path.isfile(os.path.join(docCollectionPath, f))]
    fileobj = open('frequency.txt', 'w')
    fileobj1 = open('matrix.txt', 'w')
    for eachFile in fileList:
        position = 1
        count = 0
        # docName = "Doc_Id_" + str(docId)
        # docName =  str(docId)
        docIdMap[docId] = eachFile
        lines = [line.rstrip('\n') for line in open(docCollectionPath + "/" + eachFile)]

        for eachLine in lines:
            wordList = re.split('\W+', eachLine)

            while '' in wordList:
                wordList.remove('')
            for word in wordList:
                if (word.lower() in dictionary):
                    postingList = dictionary[word.lower()]
                    if (docId in postingList):
                        postingList[docId].append(position)
                        position = position + 1
                    else:
                        postingList[docId] = [position]
                        position = position + 1
                else:
                    dictionary[word.lower()] = {docId: [position]}
                    position = position + 1
        docId = docId + 1
    #lengths = {key:len(value) for key,value in dictionary.iteritems()}
    length_dict = {key: len(value) for key, value in dictionary.items()}
    for w in length_dict:
        fileobj.write(w +"   |   "+str(length_dict[w]))
        fileobj.write("\n")
    fileobj.close()


    firstLine = "          **|**   "
    for d in docIdMap:
        firstLine = firstLine + " | " +str(d)
    fileobj1.write(firstLine+"   **|** ")
    fileobj1.write('\n')
    fileobj1.write('\n')
    for t in dictionary:
        poList = dictionary[t]
        kList = []
        for keys in poList:
            kList.append(keys)
        line = "       "

        for d in docIdMap:
            if d in kList:
                line = line + " | " + "1"
            else:
                line = line + " | " + "0"

        fileobj1.write(t+"  **|**"+line)
        fileobj1.write('\n')

    fileobj1.close()

In [4]:
def and_query(query_terms):
    if len(query_terms) == 1:
        resultList = getPostingList(query_terms[0])
        if not resultList:
            print ("")
            printString = "Result for the Query : " + query_terms[0]
            print (printString)
            print ("0 documents returned as there is no match")
            return

        else:
            print ("")
            printString = "Result for the Query : " + query_terms[0]
            print (printString)
            print ("Total documents retrieved : " + str(len(resultList)))
            for items in resultList:
                print (docIdMap[items])

    else:
        resultList = []
        for i in range(1, len(query_terms)):
            if (len(resultList) == 0):
                resultList = mergePostingList(getPostingList(query_terms[0]),
                                                    getPostingList(query_terms[i]))
            else:
                resultList = mergePostingList(resultList, getPostingList(query_terms[i]))
        print ("")
        printString = "Result for the Query(AND query) :"
        i = 1
        for keys in query_terms:
            if (i == len(query_terms)):
                printString += " " + str(keys)
            else:
                printString += " " + str(keys) + " AND"
                i = i + 1

        print (printString)
        print ("Total documents retrieved : " + str(len(resultList)))
        for items in resultList:
            print (docIdMap[items])

In [5]:
def getPostingList(term):
    if (term in dictionary):
        postingList = dictionary[term]
        keysList = []
        for keys in postingList:
            keysList.append(keys)
        keysList.sort()
        # print keysList
        return keysList
    else:
        return None

In [6]:
def mergePostingList(list1, list2):

    mergeResult = list(set(list1) & set(list2))
    mergeResult.sort()
    return mergeResult

In [7]:
def print_dict():
    # function to print the terms and posting list in the index
    fileobj = open("invertedIndex.txt", 'w')
    for key in dictionary:
        print (key + " --> " + str(dictionary[key]))
        fileobj.write(key + " --> " + str(dictionary[key]))
        fileobj.write("\n")
    fileobj.close()

In [8]:
def print_doc_list():
    for key in docIdMap:
        print ("Doc ID: " + str(key) + " ==> " + str(docIdMap[key]))

In [9]:
buildIndex()

print ("")
print ("Inverted Index :")
print_dict()

print ("")
print ("Document List :")
print_doc_list()
print ("")

QueryLines ={'THE ROAD','LAST'}

for eachLine in QueryLines:
    wordList = re.split('\W+', eachLine)

    while '' in wordList:
        wordList.remove('')
        wordsInLowerCase = []
        for word in wordList:
            wordsInLowerCase.append(word.lower())
        and_query(wordsInLowerCase)


Inverted Index :
francisco --> {1: [1, 13, 26, 30, 47, 48, 69, 94, 120, 134]}
at --> {1: [2, 61, 529, 655, 1140, 1239], 2: [499, 535, 1691, 1856], 3: [1017], 4: [190], 5: [598, 701, 859, 1032, 1399, 1467], 6: [77, 435, 440, 448, 495, 781], 7: [269, 695, 722, 837, 1018, 1396, 1633, 2067, 2309, 3113, 3259, 3639, 3817, 3964, 3984, 4345, 4951], 8: [1058, 1112], 9: [203, 494, 1003, 2802], 10: [780, 803], 11: [494, 637, 720, 1059, 1175, 1220, 1843], 14: [93, 158, 162, 184, 392, 473, 521, 562], 15: [426, 471], 16: [58, 87, 250, 257, 342, 389], 17: [132], 18: [548, 1021], 19: [464, 565, 780, 1602, 1677], 20: [217, 734, 759, 1731, 2582, 2880, 3171]}
his --> {1: [3, 294, 729, 733, 779, 851, 1227, 1240, 1258], 2: [179, 199, 248, 254, 272, 502, 767, 935, 1107, 1600, 1723, 1950, 1959, 1989, 2040, 2054], 3: [44, 131, 137, 141, 145, 153, 167, 181, 217, 224, 254, 266, 413, 493, 827, 849, 944, 1057], 4: [56, 84, 98, 114, 228, 319, 420, 609], 5: [209, 333, 352, 371, 716, 1509], 6: [37, 119, 122, 265, 5

fierce --> {1: [1003]}
events --> {1: [1004]}
harbingers --> {1: [1006]}
preceding --> {1: [1007]}
still --> {1: [1008], 2: [565, 874, 2119, 2195], 4: [666, 713], 5: [1526], 7: [353, 1623, 3461, 4084], 8: [1510], 9: [1916, 2241, 2855], 10: [469], 11: [1882], 18: [1025]}
fates --> {1: [1010], 9: [1908]}
prologue --> {1: [1012], 9: [1340, 1404, 1426], 16: [162], 20: [246]}
omen --> {1: [1015]}
coming --> {1: [1016], 7: [2737], 8: [478], 9: [816], 11: [72], 18: [917], 20: [1673]}
earth --> {1: [1021, 1116, 1248], 2: [896, 1201, 2205], 5: [733, 1319, 1356], 7: [1964, 2568], 8: [1089], 9: [1950], 15: [399], 16: [1744], 19: [1390, 1489, 1702, 1791, 1793, 1833, 2045, 2139], 20: [2340]}
demonstrated --> {1: [1023]}
climatures --> {1: [1026]}
countrymen --> {1: [1028]}
soft --> {1: [1030], 5: [467], 8: [762], 9: [3319], 10: [614], 18: [1357], 19: [1850, 1852], 20: [917]}
behold --> {1: [1031]}
lo --> {1: [1032], 7: [4030], 20: [2750]}
re --> {1: [1037], 2: [1072], 7: [514], 8: [813, 1405], 9: [

hang --> {2: [566, 1208], 18: [1518], 19: [248]}
too --> {2: [576, 1086, 1087, 1197], 3: [249, 323, 423], 4: [251], 5: [1116], 7: [297, 2155, 2342, 3093, 3258, 3405, 3409, 3962, 4186], 8: [254, 407, 431, 1208, 1223], 9: [49, 157, 394, 455, 685, 1614, 2074, 2970, 2975], 10: [215], 11: [23, 345], 15: [348], 17: [64, 145, 240], 18: [196, 697, 1036, 1612], 19: [941, 995, 1753], 20: [174, 2223, 2498, 2779, 3190]}
nighted --> {2: [587]}
colour --> {2: [588], 7: [2407, 4370], 8: [396], 11: [1153]}
friend --> {2: [597, 1388], 3: [629], 6: [386, 441], 7: [1611, 3565, 4520], 9: [1883, 1890, 2888], 16: [1155], 18: [24]}
vailed --> {2: [606]}
lids --> {2: [607]}
seek --> {2: [608], 6: [871], 7: [1915], 12: [315], 14: [11, 304, 337], 19: [1003], 20: [2699]}
noble --> {2: [611, 2080], 4: [313], 5: [320, 929], 7: [794, 2620], 8: [865, 1303, 1364], 16: [1730], 18: [39, 226], 19: [1737, 1929], 20: [2840, 3096]}
dust --> {2: [615], 7: [2665], 13: [42], 19: [1738, 1787, 1789, 2158]}
common --> {2: [620, 

galled --> {2: [1315], 9: [2176]}
wicked --> {2: [1321], 4: [361], 5: [358], 10: [518], 11: [121], 19: [2127]}
speed --> {2: [1322], 8: [1479], 14: [478], 17: [218]}
dexterity --> {2: [1327]}
incestuous --> {2: [1329], 5: [345], 10: [775], 20: [2805]}
sheets --> {2: [1330]}
tongue --> {2: [1349, 2133], 3: [488, 976], 7: [4293, 4982], 8: [1317], 9: [21, 558, 3357], 11: [113, 122, 404], 19: [632], 20: [1064]}
hail --> {2: [1356]}
lordship --> {2: [1359], 8: [895], 20: [679, 757, 797, 1032, 1380, 1487]}
glad --> {2: [1363, 1412], 7: [3554], 13: [181]}
forget --> {2: [1372], 9: [1765], 11: [991], 20: [286]}
servant --> {2: [1382], 17: [5, 15, 31]}
change --> {2: [1391], 9: [1834], 11: [1483], 20: [2592]}
faith --> {2: [1422], 5: [1095, 1194, 1205], 6: [234], 7: [2010, 3017], 9: [853, 1606], 19: [378, 1354, 1397, 1760], 20: [896, 1113, 1238]}
truant --> {2: [1429, 1466]}
disposition --> {2: [1430], 4: [463], 5: [1395], 7: [2562], 8: [110]}
enemy --> {2: [1440], 9: [1897], 20: [2006]}
ear --

pour --> {5: [511]}
leperous --> {5: [513]}
distilment --> {5: [514]}
holds --> {5: [517, 768], 7: [3030], 9: [1558], 18: [1635]}
enmity --> {5: [520]}
quicksilver --> {5: [528]}
courses --> {5: [530]}
gates --> {5: [534]}
alleys --> {5: [536]}
sudden --> {5: [543], 14: [75], 18: [418, 916], 20: [379]}
vigour --> {5: [544]}
posset --> {5: [546]}
curd --> {5: [548]}
droppings --> {5: [551]}
milk --> {5: [553]}
thin --> {5: [555]}
instant --> {5: [566, 753], 7: [4332], 17: [164]}
tetter --> {5: [567]}
bark --> {5: [568], 14: [386]}
lazar --> {5: [572]}
vile --> {5: [575], 7: [950, 955], 12: [267], 16: [920]}
loathsome --> {5: [577]}
crust --> {5: [578]}
smooth --> {5: [581], 14: [71]}
cut --> {5: [602], 18: [1109]}
blossoms --> {5: [607]}
sin --> {5: [610], 7: [3033], 16: [154]}
unhousel --> {5: [611]}
disappointed --> {5: [613]}
unanel --> {5: [614]}
reckoning --> {5: [617]}
sent --> {5: [620], 7: [544, 2349, 2387, 2416, 2477, 2510], 8: [259], 9: [2673], 14: [9], 17: [209], 19: [1282, 1

barber --> {7: [4193]}
prithee --> {7: [4198], 9: [717], 19: [1679, 2238]}
jig --> {7: [4205], 8: [1238], 9: [1073]}
bawdry --> {7: [4210]}
sleeps --> {7: [4213], 9: [2047], 13: [187], 18: [264]}
hecuba --> {7: [4218, 4690, 4693, 4699]}
mobled --> {7: [4228, 4232, 4239]}
run --> {7: [4245], 9: [1912]}
barefoot --> {7: [4246]}
threatening --> {7: [4250]}
bisson --> {7: [4254]}
rheum --> {7: [4255]}
clout --> {7: [4257]}
diadem --> {7: [4264], 11: [915]}
robe --> {7: [4269]}
lank --> {7: [4272]}
teemed --> {7: [4277]}
loins --> {7: [4278]}
blanket --> {7: [4280]}
alarm --> {7: [4283], 11: [1073]}
caught --> {7: [4286], 19: [2144]}
venom --> {7: [4295], 18: [1416], 20: [2781]}
treason --> {7: [4303], 9: [1655], 16: [993], 20: [2789, 2790]}
pronounced --> {7: [4305], 9: [14]}
sport --> {7: [4321], 9: [1958], 11: [1816]}
mincing --> {7: [4323]}
husband --> {7: [4328], 9: [1641, 1661, 1703, 1707, 1935], 11: [153, 591, 600], 16: [1116]}
clamour --> {7: [4335]}
mortal --> {7: [4341], 8: [596],

index --> {11: [511]}
counterfeit --> {11: [522]}
presentment --> {11: [523]}
brothers --> {11: [526], 19: [2327]}
seated --> {11: [532]}
curls --> {11: [538]}
front --> {11: [540]}
threaten --> {11: [549]}
herald --> {11: [556]}
mercury --> {11: [557]}
lighted --> {11: [559]}
combination --> {11: [566]}
assurance --> {11: [584], 19: [1005]}
mildew --> {11: [603]}
blasting --> {11: [606]}
mountain --> {11: [618], 16: [277], 19: [2169]}
batten --> {11: [623]}
moor --> {11: [626]}
hey --> {11: [641], 16: [1352, 1356]}
humble --> {11: [650]}
waits --> {11: [652]}
step --> {11: [660, 1010]}
apoplex --> {11: [680]}
err --> {11: [686]}
thrall --> {11: [695]}
reserved --> {11: [699]}
difference --> {11: [709], 16: [1489]}
cozen --> {11: [717]}
hoodman --> {11: [721]}
blind --> {11: [722]}
feeling --> {11: [725, 726], 19: [558]}
smelling --> {11: [734]}
sans --> {11: [735]}
mope --> {11: [749]}
mutine --> {11: [761]}
matron --> {11: [764]}
compulsive --> {11: [786]}
ardour --> {11: [787]}
fros

dove --> {16: [1371], 19: [2479]}
persuade --> {16: [1379]}
becomes --> {16: [1406], 18: [702], 20: [3468]}
steward --> {16: [1412]}
master --> {16: [1416]}
rosemary --> {16: [1429]}
pansies --> {16: [1440]}
document --> {16: [1447]}
fitted --> {16: [1453]}
fennel --> {16: [1457]}
columbines --> {16: [1461]}
rue --> {16: [1464, 1486]}
herb --> {16: [1477]}
sundays --> {16: [1480]}
daisy --> {16: [1493]}
violets --> {16: [1499], 19: [2054]}
withered --> {16: [1502]}
bonny --> {16: [1517]}
robin --> {16: [1519]}
turns --> {16: [1532]}
prettiness --> {16: [1537]}
flaxen --> {16: [1575]}
poll --> {16: [1578]}
moan --> {16: [1589]}
commune --> {16: [1621]}
collateral --> {16: [1657]}
satisfaction --> {16: [1682]}
jointly --> {16: [1698]}
due --> {16: [1706]}
obscure --> {16: [1718]}
trophy --> {16: [1721]}
hatchment --> {16: [1724]}
rite --> {16: [1731]}
formal --> {16: [1733]}
ostentation --> {16: [1734]}
axe --> {16: [1765], 19: [795], 20: [199]}
sailors --> {17: [16, 52], 18: [346]}
gree

carries --> {20: [1566]}
winnowed --> {20: [1575]}
bubbles --> {20: [1586]}
commended --> {20: [1597]}
brings --> {20: [1605]}
attend --> {20: [1611]}
constant --> {20: [1638]}
fitness --> {20: [1650]}
whensoever --> {20: [1657]}
provided --> {20: [1658]}
able --> {20: [1662]}
instructs --> {20: [1699]}
continual --> {20: [1726]}
odds --> {20: [1733, 2193, 2219]}
foolery --> {20: [1761]}
dislike --> {20: [1781]}
forestall --> {20: [1788]}
whit --> {20: [1801]}
defy --> {20: [1803]}
augury --> {20: [1804]}
sparrow --> {20: [1815]}
readiness --> {20: [1844]}
betimes --> {20: [1861]}
exception --> {20: [1940]}
denies --> {20: [1979]}
faction --> {20: [1995]}
disclaiming --> {20: [2013]}
arrow --> {20: [2032]}
hurt --> {20: [2038, 2801]}
satisfied --> {20: [2044]}
case --> {20: [2051]}
reconcilement --> {20: [2071]}
elder --> {20: [2075]}
ungored --> {20: [2092]}
embrace --> {20: [2113, 3352]}
darkest --> {20: [2153]}
weaker --> {20: [2196]}
wine --> {20: [2256]}
exchange --> {20: [2275, 2